0. open
1. review
2. topic<br>
    2.1 토픽 모델링 (Topic Modeling)<br>
3. Q & A
4. next
5. close

# 1. review

#### 카운터 기반 단어 표현
- BoW(Bag of Words) - 하나의 문서에 대한 빈도수로 단어 표현 (vector형태, 1차원)
- DTM(Document-Term Matrix) - 여러 문서에 대한 빈도수로 단어 표현 (matrix형태, 2차원)
- TF-IDF(Term Frequency-Inverse Document Frequency) : DTM에 가중치를 적용해서 단어 표현

#### 문서 유사도(Document Similarity)
- Cosine Similarity : 문서의 방향 판단
- Euclidean distance : 문서의 거리를 통한 판단
- Jaccard similarity : 합집합에서 교집합의 비율에 따른 판단

# 2. topic

## 유사도를 이용한 추천 시스템

In [1]:
import pandas as pd

#### 수집한 data를 이용하여 DataFrame 생성

In [4]:
data = pd.read_csv('the-movies-dataset/movies_metadata.csv', low_memory=False)
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


#### 20000개의 데이터로 진행

In [5]:
data = data.head(20000)

#### 'ovewview' 변수에 null data 확인 및 제거

In [7]:
data['overview'].isnull().sum()

135

In [8]:
data['overview'] = data['overview'].fillna('')

In [9]:
data['overview'].isnull().sum()

0

#### TF-IDF  수행

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

In [17]:
print(tfidf_matrix.shape)

(20000, 47487)


#### 코사인 유사도를 이용한 유사도 파악

In [18]:
from sklearn.metrics.pairwise import linear_kernel

In [19]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#### 영화 타이틀과 인덱스를 가진 테이블

In [21]:
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [23]:
idx = indices['Jumanji']
print(idx)

1


#### 선택한 영화에 대한 추천을 수행하는 함수

In [24]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title] # 선택한 영화에 대한 인덱스
    sim_scores = list(enumerate(cosine_sim[idx])) # 모든 영화에 대한 해당 영화와의 유사도
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    
    return data['title'].iloc[movie_indices]

In [25]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

In [26]:
get_recommendations('Toy Story')

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
485                      Malice
11606              Factory Girl
Name: title, dtype: object

# 2. Topic

### 2.1 Topic Modeling

- Topic은 한국어로 주제라는 의미

- Topic Modeling은 기계학습 및 자연어처리 분야에서 Topic이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델 중 하나로, text 본문의 숨겨진 의미 구조를 발견하기 위해 사용되는 Text Mining 기법

- BoW에 기반한 DTM 또는 TF-IDF는 단어의 빈도수를 이용한 수치화 방법을 활용, 단점은 의미를 고려하지 못한다는 점

- DTM의 잠재된(Latent) 의미를 이끌어내는 방법 : LSA(Latent Sematic Analysis), LDA(Latent Dirichlet Allocation)

- LSA(Latent Sematic Analysis) : 차원 축소(Dimension reduction) 기법 중 하나인 특이값 분해(Singular Value Decomposion, SVD)을 이용한 알고리즘

#### 특이값 분해(Singular Value Decomposition, SVD)

- 3개의 행렬에 대한 곱으로 분해

$A = USV^T$<br>
U : m * m 직교행렬<br>
V : n * n 직교행렬<br>
S : m * m 직사각 대각행렬<br>

- 직교행렬 : 자신과 자신의 전치행렬의 곱 또는 이를 반대로 곱한 결과가 단위행렬이 되는 행렬
- 대각행렬 : 주대각선을 제외한 곳의 원소가 모두 0인 행렬

#### 절단된 SVD(Truncated SVD) - LSA에서 사용
- Full SVD에서 일부 vector를 삭제시킨 SVD
- 절단된 SVD는 대각 행렬 S의 대각 원소의 값 중에서 상위값 t개만 남게 된다. 절단된SVD를 수행하면 값의 손실이 일어나므로 기존 행렬 A를 복구할 수 없다. U행렬과 V행렬의 t열까지만 남긴다. t는 찾고자하는 토픽의 수를 반영한 하이퍼파라미터값이다.
- 하이퍼파라미터란 사용자가 직접 값을 선택하여 성능에 영향을 주는 매개변수를 말한다.

#### LSA(Latent Sematic Analysis) 
- LSA(Latent Sematic Analysis) : 기존 DTM이나 DTM에 단어의 중요도에 따른 가중치를 주었던 TF-IDF는 단어의 의미를 고려하지 못하는 단점이 있는데 LSA에서는 이런 단점을 해소하기 위하여 절단된 SVD를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어를 적용한 알고리즘
- DTM이나 TF-IDF 행렬은 단어 의미를 고려하지 못하는 단점
- LSA는 DTM이나 TF-IDF 행렬에 절단된 SVD를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어를 적용한 알고리즘이다.

In [28]:
import numpy as np

In [30]:
A = np.array([
             [0, 0, 0, 1, 0, 1, 1, 0, 0],
             [0, 0, 0, 1, 1, 0, 1, 0, 0],
             [0, 1, 1, 0, 2, 0, 0, 0, 0],
             [1, 0, 0, 0, 0, 0, 0, 1, 1]
])
np.shape(A)

(4, 9)

#### full SVD 생성

In [31]:
U, s, VT = np.linalg.svd(A, full_matrices=True)

#### U : m * m 직교행렬

In [32]:
print(U.round(2))
np.shape(U)

[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]


(4, 4)

#### s : m * m 대각행렬

In [33]:
print(s.round(2))
np.shape(s)

[2.69 2.05 1.73 0.77]


(4,)

In [34]:
S = np.zeros((4, 9))
S[:4, :4] = np.diag(s)
print(S.round(2))
np.shape(S)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

#### Vt = n * x 직교행렬

In [35]:
print(VT.round(2))
np.shape(VT)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

#### $A = USV^T$ -> $U * S * V^T == A$

In [36]:
np.allclose(A, np.dot(np.dot(U, S), VT).round(2))

True

#### 절단된 SVD(Truncated SVD)를 생성
- t값(구하려는 topic 수)을 정하고, t값에 따른 SVD를 구성
- t = 2 (하이퍼파라미터, 임의로 설정)

In [37]:
S = S[:2, :2]
print(S.round(2))
np.shape(S)

[[2.69 0.  ]
 [0.   2.05]]


(2, 2)

In [38]:
VT = VT[:2, :]
print(VT.round(2))
np.shape(VT)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


(2, 9)

In [39]:
U = U[:, :2]
print(U.round(2))
np.shape(U)

[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


(4, 2)

In [41]:
A_prime = np.dot(np.dot(U, S), VT)
print(A)
print()
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]

[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


- U는 4 x 2의 크기를 가지고 문서 개수 x 토픽 수(t) -> 의미 파악을 원하는 개수가 2, 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터
- VT는 2 x 9의 크기를 가지고 토픽 수(t) x 단어의 개수 -> 단어의 잠재 의미를 표현하기 위해 수치화된 단어 벡터

### 문서 벡터들과 단어 벡터들을 통해 다른 문서의 유사도, 다른 단어의 유사도, 단어(Query)로 부터 문서의 유사도를 구하는 것들이 가능하다.

#### LSA를 사용해서 문서의 수를 원하는 토픽의 수로 압축한 뒤에 각 토픽당 가장 중요한 단어 5개를 출력하는 토픽 모델링 실습

In [43]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

In [45]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [46]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [47]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


#### 2. Text Preprocessing

In [48]:
news_df = pd.DataFrame({'document':documents})
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x : ' '.join([w for w in x.split() if len(w) > 3]))

news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [49]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [50]:
from nltk.corpus import stopwords

In [51]:
stop_words = stopwords.words('english')
tokenize_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenize_doc = tokenize_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [52]:
print(tokenize_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


#### 3. TF-IDF 행렬 생성

In [53]:
# 역 토큰화 (토큰화 작업을 역으로 수행)
detokenize_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenize_doc[i])
    detokenize_doc.append(t)
    
news_df['clean_doc'] = detokenize_doc

In [55]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [58]:
vectorizer = TfidfVectorizer(stop_words='english',
                             max_features=1000,
                             max_df=0.5,
                             smooth_idf=True)

In [59]:
X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape

(11314, 1000)

#### 4. Topic Modeling

In [60]:
from sklearn.decomposition import TruncatedSVD

In [61]:
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [62]:
np.shape(svd_model.components_) # 토픽수 t X 단어 수 : 단어 벡터

(20, 1000)

In [65]:
terms = vectorizer.get_feature_names()

def get_topics(components,feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic {0:2}: {1}".format((idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]]))
        
get_topics(svd_model.components_, terms)

Topic  1: [('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic  2: [('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic  3: [('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic  4: [('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic  5: [('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic  6: [('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic  7: [('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic  8: [('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic  9: [('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]

#### Topic Modeling은 문서의 집합에서 topic을 찾아내는 프로세스이다. 검색 엔진, 고객 민원 시스템 등과 같이 문서의 주제를 알아내는 일이 중요한 곳에서 사용된다.

#### 잠재 디리클레 할당(Letent Dirichlet Allocation, LDA)은 topic modeling의 대표적인 알고리즘이다.

#### LDA는 문서들은 topic들의 혼합으로 구성되어져 있으며, topic들은 확률 분호에 기반하여 단어들을 생성한다고 가장한다.

- 데이터가 주어지면 LDA는 문서가 생성되던 과정을 역추적한다.

#### LSA와 LDA의 차이
- LSA : DTM을 차원 축소하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.
- LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다

#### LDA를 이용한 topic modeling 전체 과정

1) LDA를 수행하기 위해 우선 k값(t값, 하이퍼파라미터, topic 수) 결정
2) LDA가 각 문서의 topic 분포와 각 topic 내의 단어 분폴르 추정한다.

< 각 문서 topic 분포 >
- 문서1 : 토픽 A 100%
- 문서2 : 토픽 B 100%
- 문서3 : 토픽 B 60%, 토픽 A 40%

#### LDA의 가정
- LDA는 문서의 집합으로부터 어떤 topic이 존재하는지를 알아내기 위한 알고리즘이다.
- LDA는 DTM/TF-IDF 행렬을 입력으로 하여 topic을 알아내는 알고리즘으로 단어의 순서는 고려하지 않는다.
- LDA는 문서의 집합으로부터 topic을 뽑아낼 때 다음과 같은 가정을 염두에 두고 수행한다. " 나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을거야."

1. 문서에 사용할 단어의 개수 N을 정한다.
2. 문서에 사용할 topic의 혼합을 확률 분포에 기반하여 결정한다.
3. 문서에 사용할 각 단어를 다음과 같이 정한다<br>
    3.1 topic 분포에서 topic T를 확률적으로 고른다.<br>
    3.2 선택한 topic T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고른다.
   
이러한 과정을 통해 문서가 작성되었다는 가정 하에 LDA는 topic을 뽑아내기 위하여 위 과정을 역으로 추적하는 역공학(reverse engineering)을 수행한다.

#### LDA 수행 과정
1. 사용자는 알고리즘에게 topic의 개수 k를 알려준다
2. 모든 문서를 k개 중 하나의 topic에 할당한다.
3. 이제 모든 문서의 모든 단어에 대해서 다음 사항을 반복해서 진행한다.(iteration)<br>
    3.1 어떤 문서의 각 단어 w는 자신은 잘못된 topic에 할당되어져 있지만, 다른 단어들은 전부 올바른 topic에 할당되어져 있는 상태라고 가정한다. 이에 따라 단어 w는 다음 기준에 따라 topic이 재할당된다.
    
    p(topic | document d) : 문서 d의 단어들 중 topic t에 해당하는 단어들의 비율<br>
    p(word w | topic t) : 단어 w를 갖고 있는 모든 문서들 중 topic t가 할당된 비율

### LDA 실습

In [66]:
tokenize_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [67]:
from gensim import corpora

#### 각 단어에 대한 정수 인코딩 수행 - (word_id, word_frequency)

In [68]:
dictionary = corpora.Dictionary(tokenize_doc)
corpus = [dictionary.doc2bow(text) for text in tokenize_doc]
print(corpus[1])

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [69]:
print(dictionary[52])

well


In [70]:
print(dictionary[66])

faith


In [71]:
len(dictionary)

64281

#### LDA 알고리즘을 이용한 Topic modeling

In [73]:
import gensim

In [74]:
num_topics = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=4)

for topic in topics:
    print(topic)

(0, '0.036*"space" + 0.013*"nasa" + 0.008*"launch" + 0.008*"earth"')
(1, '0.013*"health" + 0.010*"medical" + 0.007*"rate" + 0.007*"pain"')
(2, '0.038*"period" + 0.021*"power" + 0.016*"play" + 0.015*"scorer"')
(3, '0.018*"windows" + 0.015*"thanks" + 0.012*"know" + 0.012*"anyone"')
(4, '0.019*"would" + 0.013*"people" + 0.011*"think" + 0.010*"know"')
(5, '0.014*"available" + 0.011*"information" + 0.010*"mail" + 0.009*"also"')
(6, '0.027*"food" + 0.010*"diet" + 0.008*"prophecy" + 0.007*"chinese"')
(7, '0.036*"window" + 0.021*"motif" + 0.018*"widget" + 0.018*"server"')
(8, '0.018*"game" + 0.017*"team" + 0.014*"year" + 0.013*"games"')
(9, '0.009*"government" + 0.009*"public" + 0.008*"system" + 0.007*"encryption"')
(10, '0.085*"drive" + 0.035*"disk" + 0.030*"hard" + 0.030*"drives"')
(11, '0.037*"file" + 0.023*"output" + 0.019*"entry" + 0.017*"program"')
(12, '0.049*"israel" + 0.031*"israeli" + 0.023*"jews" + 0.020*"arab"')
(13, '0.024*"jesus" + 0.015*"christian" + 0.014*"bible" + 0.012*"churc

In [75]:
print(ldamodel.print_topics())

[(0, '0.036*"space" + 0.013*"nasa" + 0.008*"launch" + 0.008*"earth" + 0.007*"satellite" + 0.006*"shuttle" + 0.005*"orbit" + 0.005*"station" + 0.005*"lunar" + 0.005*"mission"'), (1, '0.013*"health" + 0.010*"medical" + 0.007*"rate" + 0.007*"pain" + 0.007*"disease" + 0.006*"study" + 0.006*"among" + 0.005*"doctor" + 0.005*"research" + 0.005*"patients"'), (2, '0.038*"period" + 0.021*"power" + 0.016*"play" + 0.015*"scorer" + 0.009*"third" + 0.009*"second" + 0.008*"murphy" + 0.008*"brown" + 0.007*"first" + 0.006*"recchi"'), (3, '0.018*"windows" + 0.015*"thanks" + 0.012*"know" + 0.012*"anyone" + 0.011*"card" + 0.011*"would" + 0.011*"system" + 0.010*"please" + 0.009*"problem" + 0.009*"like"'), (4, '0.019*"would" + 0.013*"people" + 0.011*"think" + 0.010*"know" + 0.010*"like" + 0.007*"even" + 0.007*"time" + 0.006*"could" + 0.006*"well" + 0.006*"much"'), (5, '0.014*"available" + 0.011*"information" + 0.010*"mail" + 0.009*"also" + 0.008*"list" + 0.008*"software" + 0.008*"send" + 0.007*"version" + 0

### 실습

https://serieson.naver.com/movie/recommendList.nhn

위 site에서 영화별 줄거리를 crawling 수행 후 파일에 저장하고 DTM/TF-IDF, 문서 유사도, Topic modeling을 적용하기